In [25]:
# Google Driveと接続を行います。これを行うことで、Driveにあるデータにアクセスできるようになります。
# 下記セルを実行すると、Googleアカウントのログインを求められますのでログインしてください。
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
!pip install ipywidgets

In [27]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import joblib
from tensorflow.keras.models import load_model


# ダミーデータの生成
np.random.seed(42)
data = {
    'feature1': np.random.randint(0, 24, 100),  # 時間帯 (0 - 23 時)
    'feature2': np.random.rand(100) * 200,  # 車両の速度 (0 - 200 km/h)
    'feature3': np.random.randint(0, 4, 100),  # 天候 (0: 晴れ, 1: 曇り 2: 雨, 3: 雪)
    'feature4': np.random.rand(100) * (40 - (-42)) + (-42),# 気温 (-42 ～ 40°C)
    'feature5': np.random.rand(100) * 255,  # 風速 (0  ～ 255 km/h)
    'feature6': np.random.randint(0, 3, 100),  # 路面状況 (0: 良好, 1: 濡れている, 2: 凍結)
    'accident': np.random.randint(0, 2, 100)  # 事故 (0: なし, 1: あり)
}

df = pd.DataFrame(data)

# 特徴量とターゲットの分割
X = df.drop('accident', axis=1)
y = df['accident']

# データの標準化
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 訓練データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# モデルの構築
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# モデルのコンパイル
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# モデルの訓練
model.fit(X_train, y_train, epochs=50, batch_size=10, validation_data=(X_test, y_test))


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - accuracy: 0.4691 - loss: 0.7175 - val_accuracy: 0.4500 - val_loss: 0.7229
Epoch 2/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5288 - loss: 0.6911 - val_accuracy: 0.4500 - val_loss: 0.7247
Epoch 3/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4894 - loss: 0.7021 - val_accuracy: 0.4500 - val_loss: 0.7217
Epoch 4/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5460 - loss: 0.6781 - val_accuracy: 0.4000 - val_loss: 0.7208
Epoch 5/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6282 - loss: 0.6652 - val_accuracy: 0.3500 - val_loss: 0.7251
Epoch 6/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.6552 - loss: 0.6571 - val_accuracy: 0.3000 - val_loss: 0.7336
Epoch 7/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7046 - loss: 0.6282 - val_accuracy: 0.3500 - val_loss: 0.7349
Epoch 8/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6929 - loss: 0.6350 - val_accuracy: 0.3500 - val_loss: 0.7339
Epoch 9/50
8

In [28]:
# 警告を無効にする
import logging
import absl.logging

absl.logging.set_verbosity(absl.logging.ERROR)
logging.getLogger('absl').setLevel(logging.ERROR)


In [29]:
# モデルとスケーラーの保存
model.save('/content/drive/My Drive/ディープラーニングアプリ/事故予測アプリ2/accident_model.h5')
joblib.dump(scaler, '/content/drive/My Drive/ディープラーニングアプリ/事故予測アプリ2/scaler.pkl')


['/content/drive/My Drive/ディープラーニングアプリ/事故予測アプリ2/scaler.pkl']

In [30]:
# モデルとスケーラーのロードとコンパイルを一度だけ行う
model = load_model('/content/drive/My Drive/ディープラーニングアプリ/事故予測アプリ2/accident_model.h5')
scaler = joblib.load('/content/drive/My Drive/ディープラーニングアプリ/事故予測アプリ2/scaler.pkl')
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



In [31]:
import ipywidgets as widgets
from IPython.display import display


def predict_accident(input_data):
    # 入力データの標準化
    input_data_scaled = scaler.transform(input_data)

    # 予測
    prediction = model.predict(input_data_scaled)
    return prediction


# 事故予測関数
def predict(button):
    try:
        # 各入力値を取得
        time = float(entry1.value)
        speed = float(entry2.value)
        weather = float(entry3.value)
        temperature = float(entry4.value)
        wind_speed = float(entry5.value)
        road_condition = float(entry6.value)

        # 入力値の検証
        if not (0 <= time < 24):
            raise ValueError("時間帯は0から23の範囲でなければなりません。")
        if not (0 <= speed <= 200):
            raise ValueError("車両の速度は0から200 km/hの範囲でなければなりません。")
        if not (0 <= weather <= 3):
            raise ValueError("天候の値は0から3の範囲でなければなりません。")
        if not (-42 <= temperature <= 40):
            raise ValueError("気温は-42°Cから40°Cの範囲でなければなりません。")
        if not (0 <= wind_speed <= 255):
            raise ValueError("風速は0から255 km/hの範囲でなければなりません。")
        if not (0 <= road_condition <= 2):
            raise ValueError("路面状況の値は0から2の範囲でなければなりません。")

        input_data = np.array([
            time,
            speed,
            weather,
            temperature,
            wind_speed,
            road_condition
        ]).reshape(1, -1)

        # フィーチャー名を含むデータフレームに変換
        input_data_df = pd.DataFrame(input_data, columns=['feature1', 'feature2', 'feature3', 'feature4','feature5','feature6'])

        # データの標準化
        input_data_scaled = scaler.transform(input_data_df)

        # モデルの予測
        prediction = model.predict(input_data_scaled)[0][0]

        # 入力条件の判定
        is_input_condition_1 = (
            weather == 0 and
            8 <= time < 17 and
            speed <= 49 and
            1 <= temperature <= 34 and
            wind_speed <= 19 and
            road_condition == 0
        )

        is_input_condition_2 = (
            weather == 2 and
            7 <= time < 8 and
            speed >= 250 and
            (temperature <= 0 or temperature >= 35) and
            wind_speed >= 20 and
            road_condition == 1
        )

        # ダミーデータの条件を仮定（例として）
        is_dummy_condition_1 = (
            weather == 1 and
            19 <= time < 7 and
            speed <= 49 and
            1 <= temperature <= 34 and
            wind_speed <= 19 and
            road_condition == 0
        )

        is_dummy_condition_2 = (
            weather == 3 and
            17 <= time < 19 and
            speed >= 250 and
            (temperature <= 0 or temperature >= 35) and
            wind_speed >= 20 and
            road_condition == 2
        )

        # 条件に基づいた予測結果の制御
        if is_input_condition_1 and is_dummy_condition_1:
            result.value = "事故が発生する可能性は低いです。"
        elif (is_input_condition_1 and is_dummy_condition_2) or (is_input_condition_2 and is_dummy_condition_1) or (is_input_condition_2 and is_dummy_condition_2):
            result.value = "事故が発生する可能性があります。"
        else:
            if prediction > 0.5:
                result.value = "事故が発生する可能性があります。"
            else:
                result.value = "事故が発生する可能性は低いです。"

    except ValueError as ve:
        result.value = f"入力エラー: {ve}"
    except Exception as e:
        result.value = f"予測中にエラーが発生しました: {e}"


# 入力フィールド
entry1 = widgets.FloatText(layout=widgets.Layout(width='200px'))
entry2 = widgets.FloatText(layout=widgets.Layout(width='200px'))
entry3 = widgets.FloatText(layout=widgets.Layout(width='200px'))
entry4 = widgets.FloatText(layout=widgets.Layout(width='200px'))
entry5 = widgets.FloatText(layout=widgets.Layout(width='200px'))
entry6 = widgets.FloatText(layout=widgets.Layout(width='200px'))


# テキストラベル
label1 = widgets.Label(value='時間帯 (0 ～ 23 時):', layout=widgets.Layout(width='300px'))
label2 = widgets.Label(value='車両の速度 (0 ～ 200 km/h):', layout=widgets.Layout(width='300px'))
label3 = widgets.Label(value='天候 (0: 晴れ　1: 曇り　2: 雨　3: 雪):', layout=widgets.Layout(width='300px'))
label4 = widgets.Label(value='気温 (-42 ～ 40°C):', layout=widgets.Layout(width='300px'))
label5 = widgets.Label(value='風速 (0  ～ 255 km/h):', layout=widgets.Layout(width='300px'))
label6 = widgets.Label(value='路面状況 (0: 良好　1: 濡れている　2: 凍結):', layout=widgets.Layout(width='300px'))

# 各行のレイアウト
row1 = widgets.HBox([label1, entry1], layout=widgets.Layout(align_items='flex-start'))
row2 = widgets.HBox([label2, entry2], layout=widgets.Layout(align_items='flex-start'))
row3 = widgets.HBox([label3, entry3], layout=widgets.Layout(align_items='flex-start'))
row4 = widgets.HBox([label4, entry4], layout=widgets.Layout(align_items='flex-start'))
row5 = widgets.HBox([label5, entry5], layout=widgets.Layout(align_items='flex-start'))
row6 = widgets.HBox([label6, entry6], layout=widgets.Layout(align_items='flex-start'))


# 予測ボタン
predict_button = widgets.Button(description="予測する")
predict_button.on_click(predict)  # ボタンにイベントリスナーを追加

# 結果表示
result = widgets.Label(value="")

# 全体のレイアウト
display(row1, row2, row3, row4,row5,row6, predict_button, result)

Button(description='予測する', style=ButtonStyle())

Label(value='')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━